## Demo 1: produce lagrangian particule trajectories  from eNATL60 with Ocean PArcels

* This is a demo notebook of using thw software Ocean Parcels (https://oceanparcels.org/Lagrangian) to produce lagrangian trajectories from the eNATL60 1-h currents (u,v at 15 m depth).


#### Objective of this notebook:
* Initialize 146 particles, evenly spaced over the western mediterranean domain. 
* Advect for 1 month and save outputs.
* 1h output frequency
* Use 15-meter depth currents (u,v) from eNATL60-no-tide.
* Sample u, v along the trajectory.
* Restart another experiment from previous outputs if needed.


#### Misc. notes:
* An extraction of the eNatl60 data over the Western Mediterranean region has been applied ahead of this notebook. We are loading these extracted files below.
* A 1-month advection experiment -> 20 min and no memory problems (the number of particles or the addition of u,v sampled fields don't change the computing time much).
* I noticed that running a second experiment in  the same open notebook leads to memory problems (some sort of memory leakage?) on Jean Zay. So each 1-month experiment should be run in a newly opened notebook (shutdown in between experiments).
* u,v sampled along the trajectories are initially in rad/sec and must be converted to m/s in the end (see at the end of this notebook)

#### Machine:
* This notebook has run on the french HPCJeanZay@IDRIS, on the prepost partition (FREE CPU up to 2h runtime, and more memory per node)
    ``` 
    salloc --account egi@cpu --ntasks=1 --cpus-per-task 10 --hint=nomultithread --partition prepost   --time=03:00:00 srun --pty bash 
    ```
    Then:
    ```
    module load parcels/2.2.1
    idrjup
    ```

## Imports

In [1]:
## standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# plot
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


# custom tools for plotting
import lib_medwest60 as slx


# for ocean parcels
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, ParticleFile,ErrorCode
from argparse import ArgumentParser
import numpy as np
import pytest
from glob import glob
from datetime import timedelta as delta
from os import path
import time
from netCDF4 import Dataset


# for jupyter notebook display
%matplotlib inline 

# Prep step:

In [4]:
# region to process (MEDWEST in our case)
rbox='MEDWEST'

# experiment number
exp=2

# new particle set or restarted from existing particle file
newset=True

# output directory for plots and drifters file
diro="/gpfswork/rech/egi/regi915/PLT/EUROSEA_DRIFTERS/sandrine/prod/exp2/"

# input directory (for eNATL60 data files)
data_path = '/gpfsstore/rech/egi/commun/EUROSEA/eNATL60-BLB002/'


# u,v,ssh files
ufiles = sorted(glob(data_path+'eNATL60'+rbox+'-BLB002_y2009m*.1h_vozocrtx_15m.nc'))
vfiles = sorted(glob(data_path+'eNATL60'+rbox+'-BLB002_y2009m*.1h_vomecrty_15m.nc'))
sshfiles = sorted(glob(data_path+'eNATL60'+rbox+'-BLB002_y2009m*.1h_sossheig.nc'))

# grid files
meshfi = 'mesh_hgr_eNATL60'+rbox+'_3.6.nc'
maskfile   =data_path+'/mask_eNATL60'+rbox+'_3.6.nc'

---

# Step 1:  Ocean Parcels reads the model data (and creates the "fieldset")

In [8]:

# now creates the Ocean PArcels "fieldset" :
# (see Ocean Parcels docs: 
# https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_curvilinear.ipynb)
filenames = {'U': {'lon': data_path + meshfi,
                   'lat': data_path + meshfi,
                   'data': ufiles},
             'V': {'lon': data_path + meshfi,
                   'lat': data_path + meshfi,
                   'data': vfiles}  }  #,
           # 'SSH': {'lon': data_path + meshfi,
           #        'lat': data_path + meshfi,
           #        'data': sshfiles},}

variables = {'U': 'vozocrtx', 'V': 'vomecrty'}#, 'SSH': 'sossheig'}

dimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}

# Fieldset definition:
field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=False)

---

# Step 2:  Create the particle set

In [9]:
# output and restart file names (filo and filorst)
filo=rbox+"-EUROSEA_146p_30d_freq1h_start_10-2009_exp"+str(exp)
 # idlen is the number of days in the month over which to run the experiment, basically 30 or 31 (or 28)
idlen=31


* Define a new particle class where u and v are also sampled* 

In [12]:
class SampleParticle(JITParticle):         # Define a new particle class where u and v are also sampled
    #ssh = Variable('ssh')  
    uinterp = Variable('uinterp')  
    vinterp = Variable('vinterp')    

* create initial, evenly spaced, set of particle locations:

Here we pick up evenly spaced (longitudes,latitudes) from the model grid and excluding land points.
To increase the coverage, increase the increment incr

In [13]:
if (seg == 0):
    # pick up lon lat from the model grid (evenly spaced) in the ocean (excluding land points)
    incr=45
    nlatp = nav_lat_refU.where(((maskU>0.)&(maskV>0.)))[8:803:incr,2:883:incr]
    nlonp = nav_lon_refU.where(((maskU>0.)&(maskV>0.)))[8:803:incr,2:883:incr]

    # make sure we are excuding the same points in both arrays
    nlatp = nlatp.where(nlonp)
    nlonp = nlonp.where(nlatp)

    # stack x, y in one dimension
    nlatp_st = nlatp.stack(z=("x", "y"))
    nlonp_st = nlonp.stack(z=("x", "y"))

    # drop nan values to get a list of lon lat to feed parcels
    nlatp_st_nona = nlatp_st.dropna(dim='z')
    nlonp_st_nona = nlonp_st.dropna(dim='z')

    # print size (i.e. number of particles)
    print(nlatp_st_nona.size)

146


* create particle set (either new evenly spaced locations or restarted from previous file)

In [14]:
if newset:
    # IF NEW PARTICLE SET:
    # Create the particle set by sampling the field set according to the lat,lon list created above
    npart = nlatp_st_nona.size  # number of particles to be released
    td = xr.full_like(nlatp_st_nona,delta(days=31+31+30).total_seconds())   # release every particle one hour later
    pset = ParticleSet.from_list(fieldset=field_set, pclass=SampleParticle, lon=nlonp_st_nona, lat=nlatp_st_nona,time=td)

else:
    # IF RESTART FROM EXISTING PARTICLE SET:
    pset = ParticleSet.from_particlefile(fieldset=field_set, pclass=SampleParticle, filename=diro+filorst+'.nc', restart=True)

---

# Step 3:  Advect the particle set

* customization functions:

In [15]:
# Custom function that samples U and V at particle location
def SampleUV(particle, fieldset, time):  
    (u1, v1) = fieldset.UV[time, particle.depth,particle.lat, particle.lon]
    particle.uinterp = u1
    particle.vinterp = v1
    
# create a function that will kill particles when they go out of the regional boundaries
# see https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_Agulhasparticles.ipynb
def DeleteParticle(particle, fieldset, time):
    particle.delete()

* set kernels to apply to the particle set:

In [16]:
# basic RK4 advection kernel
k_ADVRK4 = pset.Kernel(AdvectionRK4)

# additional kernel to sample u,v    
k_sampleUV = pset.Kernel(SampleUV)    # Casting the SampleP function to a kernel.

* advect the particles for 1 month:

In [ ]:
# start time of the computation idlen
start = time.time()

# define the characteristics of the particle file (the trajectories)
pfile = ParticleFile(diro+filo, pset, outputdt=delta(hours=1))

# compute with the kernel defined above
#pset.execute(k_ADVRK4+k_sampleSSH+k_sampleUV, runtime=delta(days=6), dt=delta(hours=0.25), output_file=pfile)
pset.execute(k_ADVRK4+k_sampleUV, runtime=delta(days=idlen), dt=delta(hours=1), output_file=pfile,recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

# display time of computation
end = time.time()
print(end-start)

# save output to netcdf
pfile.export()

INFO: Compiled SampleParticleAdvectionRK4SampleUV ==> /tmp/parcels-43033/f72a8274eaa5c5462153af608437d190_0.so
INFO: Temporary output files are stored in /gpfswork/rech/egi/regi915/PLT/EUROSEA_DRIFTERS/sandrine/prod/exp2/out-RQKUFQME.
INFO: You can use "parcels_convert_npydir_to_netcdf /gpfswork/rech/egi/regi915/PLT/EUROSEA_DRIFTERS/sandrine/prod/exp2/out-RQKUFQME" to convert these to a NetCDF file during the run.
 59% (1594800.0 of 2678400.0) |#####     | Elapsed Time: 0:05:08 ETA:   0:08:52

---

## Step 4: s|ee next notebooks to convert and plot the resuts